## 使用GPT_API_free获得密钥

In [1]:
import getpass
import os
import langchain
from langchain.schema.messages import HumanMessage, SystemMessage,AIMessage
from langchain_openai import ChatOpenAI

from openai import OpenAI
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key="OPENAI-API-KEY",
    # base_url=https://api.chatanywhere.tech/v1"
)
model = ChatOpenAI(model="gpt-3.5-turbo")

In [2]:
messages = [
    SystemMessage(content="用中文回复"),
    HumanMessage(content="中国有几个省份"),
]
model.invoke(messages)

AIMessage(content='中国共有34个省级行政区域，包括23个省、5个自治区、4个直辖市和2个特别行政区。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 23, 'total_tokens': 63, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_0165350fbb', 'finish_reason': 'stop', 'logprobs': None}, id='run-986f2cdd-133a-449d-8470-2ec883814df1-0', usage_metadata={'input_tokens': 23, 'output_tokens': 40, 'total_tokens': 63, 'input_token_details': {}, 'output_token_details': {}})

# 构建一个代理

#### 在 LangChain 中，Agent 是一种强大的工具，能够自动化决策过程，选择并使用不同的工具来处理给定的任务。LangChain 的代理通过与LLM结合，可以进行多步骤的任务处理，并能动态地根据上下文和任务需求选择合适的工具或操作，从而完成更复杂的任务。

下面的代码片段表示一个完全功能的代理，它使用LLM来决定使用哪些工具，配备了一个通用搜索工具，并且具有对话记忆。-这意味着它可以作为一个多轮聊天机器人使用。

In [3]:
%pip install langchain langchain-anthropic langchain-community

Looking in indexes: https://mirrors.tuna.tsinghua.edu.cn/pypi/web/simple
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install -U langchain-community langgraph langchain-anthropic tavily-python langgraph-checkpoint-sqlite

Looking in indexes: https://mirrors.tuna.tsinghua.edu.cn/pypi/web/simple
Note: you may need to restart the kernel to use updated packages.


#### 使用 Tavily（一个搜索引擎）作为工具。 使用他需要获取并设置一个API密钥：

In [5]:
os.environ["TAVILY_API_KEY"] = "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"

In [6]:
from langchain_community.tools.tavily_search import TavilySearchResults #Tavily 是一个用于语义搜索的工具，可以根据查询返回相关的搜索结果。
search = TavilySearchResults(max_results=1)#创建一个 TavilySearchResults 实例 search，设置每次搜索时最多返回 1 个结果。max_results 参数控制返回结果的数量，以避免过多的返回项。
search_results = search.invoke("what is the weather in Chongqing")
print(search_results)

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Chongqing', 'region': 'Chongqing', 'country': 'China', 'lat': 29.5628, 'lon': 106.5528, 'tz_id': 'Asia/Shanghai', 'localtime_epoch': 1736745843, 'localtime': '2025-01-13 13:24'}, 'current': {'last_updated_epoch': 1736745300, 'last_updated': '2025-01-13 13:15', 'temp_c': 9.0, 'temp_f': 48.2, 'is_day': 1, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/day/143.png', 'code': 1030}, 'wind_mph': 2.9, 'wind_kph': 4.7, 'wind_degree': 274, 'wind_dir': 'W', 'pressure_mb': 1022.0, 'pressure_in': 30.18, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 87, 'cloud': 0, 'feelslike_c': 8.7, 'feelslike_f': 47.7, 'windchill_c': 13.5, 'windchill_f': 56.4, 'heatindex_c': 13.3, 'heatindex_f': 55.9, 'dewpoint_c': 3.6, 'dewpoint_f': 38.5, 'vis_km': 3.5, 'vis_miles': 2.0, 'uv': 3.1, 'gust_mph': 3.4, 'gust_kph': 5.4}}"}]


## 使用语言模型

In [7]:
import getpass
import os
from langchain_openai import ChatOpenAI

api_key="OPENAI-API-KEY",
model = ChatOpenAI(model="gpt-3.5-turbo")

#### 通过传入消息列表来调用语言模型。默认情况下，响应是一个 content 字符串。

In [8]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

#### 现在可以看到启用此模型进行工具调用的情况。为了启用这一点，使用 .bind_tools 来让语言模型了解这些工具。

In [10]:
from langchain.tools import Tool
tools = [
    Tool(
        name="WeatherTool",  # 工具名称应符合规则：字母、数字、下划线或短横线
        func=lambda location: f"The weather in {location} is sunny.",  # 工具功能
        description="Returns weather information for a given location."
    ),
    Tool(
        name="MathTool",
        func=lambda x: str(eval(x)),  # 进行简单的数学运算
        description="Performs basic math operations."
    )
]

# 将工具绑定到模型
model_with_tools = model.bind_tools(tools)

#### 现在可以调用模型。我们先用一条普通消息调用它，看看它的响应。我们可以查看 content 字段和 tool_calls 字段。

In [11]:
response = model_with_tools.invoke([HumanMessage(content="What is AI?")])

print(f"ContentString: {response.content}") #打印模型的回应内容
print(f"ToolCalls: {response.tool_calls}") #显示模型是否使用了外部工具，以及调用工具的详细信息

ContentString: AI, or artificial intelligence, refers to the simulation of human intelligence processes by machines, especially computer systems. These processes include learning, reasoning, problem-solving, perception, and language understanding. AI is used in various applications such as speech recognition, natural language processing, computer vision, and decision-making.
ToolCalls: []


#### 我们可以看到现在没有文本内容，但有一个工具调用。它希望我们调用 Tavily Search 工具。但是这还没有调用那个工具，它只是告诉我们去调用。为了实际调用它，需要创建我们的代理。

## 创建代理

#### 现在已经定义了工具和大型语言模型，我们可以创建代理。使用 LangGraph 来构建代理。 目前正在使用高级接口来构建代理，但LangGraph的好处在于，这个高级接口是由一个低级的、高度可控的API支持的，以防想要修改代理逻辑。现在，可以用大型语言模型和工具初始化代理。请注意，我们传入的是 model，而不是 model_with_tools。这是因为 create_react_agent 会在后台为我们调用 .bind_tools

In [12]:
from langgraph.prebuilt import create_react_agent 

agent_executor = create_react_agent(model, tools)#创建一个反应式代理 React Agent 的函数

## 运行代理

现在可以在几个查询上运行代理。请注意，目前这些都是无状态查询（它不会记住之前的交互），代理将在交互结束时返回最终状态（包括任何输入）。当不需要调用工具时它的响应：

In [13]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]}) #
response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='aecbbd8d-2aa0-45c0-afec-fb01e2dcfd9d'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 84, 'total_tokens': 94, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_808245b034', 'finish_reason': 'stop', 'logprobs': None}, id='run-92c9a60d-7145-4fbd-ac00-ac48c44fbcdb-0', usage_metadata={'input_tokens': 84, 'output_tokens': 10, 'total_tokens': 94, 'input_token_details': {}, 'output_token_details': {}})]

让我们在一个应该调用工具的示例上试试：在这个过程中，模型不仅生成文本响应，还可能触发工具（如天气查询工具）来获取所需的数据。

In [14]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in Chongqing?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in Chongqing?', additional_kwargs={}, response_metadata={}, id='5ee68acd-1842-4b09-a61e-666b1f346b8b'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_zT4m7EqWkvoCQFluEmva4S01', 'function': {'arguments': '{"__arg1":"Chongqing"}', 'name': 'WeatherTool'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 91, 'total_tokens': 110, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_0165350fbb', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-56b53812-594a-44c3-a16d-af711e971307-0', tool_calls=[{'name': 'WeatherTool', 'args': {'__arg1': 'Chongqing'}, 'id': 'call_zT4m7EqWkvoCQFluEmva4S01', 'type': 'tool_call'}], usage_metadata={'input_tokens': 91, 'output_tokens': 19, 'total_tokens': 110, 'input_token_details': {}, 'output_token_details': {}}),
 ToolMessage(content='The weather in C

## 流式消息

#### 上面已经看到如何使用 .invoke 调用代理以获取最终响应。如果代理正在执行多个步骤，这可能需要一些时间。为了显示中间进度，我们可以在消息发生时流式返回消息

In [15]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_iZuYTjZrRHW0JymUIQx7jRCd', 'function': {'arguments': '{"__arg1":"San Francisco"}', 'name': 'WeatherTool'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 89, 'total_tokens': 107, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_44b923fd7f', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-08f74809-8554-43f0-a739-e71c0b2686b7-0', tool_calls=[{'name': 'WeatherTool', 'args': {'__arg1': 'San Francisco'}, 'id': 'call_iZuYTjZrRHW0JymUIQx7jRCd', 'type': 'tool_call'}], usage_metadata={'input_tokens': 89, 'output_tokens': 18, 'total_tokens': 107, 'input_token_details': {}, 'output_token_details': {}})]}}
----
{'tools': {'messages': [ToolMessage(content='The weather in San Francisco is sunny.', name='WeatherTool', id='4404fcd6-5f1c-4312-8a28-2660ec6c341

### 流式令牌

#### 除了流式返回消息，流式返回令牌也是有用的。 可以使用 .astream_events 方法来实现这一点

In [16]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="whats the weather in SF?")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

--
Starting tool: WeatherTool with inputs: {'__arg1': 'San Francisco'}
Done tool: WeatherTool
Tool output was: content='The weather in San Francisco is sunny.' name='WeatherTool' tool_call_id='call_T1lMVI6i2iZriaHIGch3k6MV'
--
The| weather| in| San| Francisco| is| sunny|.|

### 添加内存

#### 如前所述，该代理是无状态的。这意味着它不记得之前的交互。为了给它添加内存，我们需要传入一个检查点。当传入检查点时，我们还必须在调用代理时传入 thread_id（以便它知道从哪个线程/对话恢复）。

In [17]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [18]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [19]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im baby!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 86, 'total_tokens': 96, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_808245b034', 'finish_reason': 'stop', 'logprobs': None}, id='run-e7ffd7bb-bb06-4b9b-ac29-f1bc8f0c6d1a-0', usage_metadata={'input_tokens': 86, 'output_tokens': 10, 'total_tokens': 96, 'input_token_details': {}, 'output_token_details': {}})]}}
----


In [20]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="I'm sorry, but I don't have access to your personal information. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 108, 'total_tokens': 131, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_0165350fbb', 'finish_reason': 'stop', 'logprobs': None}, id='run-bf20548f-9dbb-4c0b-9ddb-825f5f09a44e-0', usage_metadata={'input_tokens': 108, 'output_tokens': 23, 'total_tokens': 131, 'input_token_details': {}, 'output_token_details': {}})]}}
----
